In [1]:
from pims import ImageSequence
import numpy as np
import skimage.exposure as skie

# Imports all .tif files in a directory as float32
# Pass file header (fh) and file extension (fe)
# Needs pims and numpy
_
def seq_import(fh, fe):
    return ImageSequence(fh + '*' + fe, dtype = np.float32)
# Import and rescale image sequence to run between 1 and 65536
# Pass file header (fh) and file extension (fe)
def im_rescale(fh, fe):
    imseq = seq_import(fh,fe)
    n = len(imseq)
    o = []
    for i in range (0, n):
        o.append(skie.rescale_intensity(imseq[i], in_range = (1, 65536)))
    return o

In [4]:
import skimage.exposure as skie
# Last frame
lf = seq_import('Melt_Kinetic_Run_18_127_end-', '.tif')
# Histogram equalization of last frame
him = skie.exposure.equalize_hist(lf[0])
# Rescale image in case blob detection fails
shim = skie.exposure.rescale_intensity(him)

In [6]:
import matplotlib.pyplot as plt
plt.imshow(him, cmap = 'BuGn')
plt.show()

In [ ]:
'''
fn = 'Melt_Kinetic_Run_9_157-' # File name header
ext = '.tif' # File extension

# Import sequence of images 
imseq = ImageSequence(fn + '*' + ext, dtype = np.float32)

# Create empty list for normalized images (pixel scale values to start at 1)
im_norm = [];
# Normalize list of n images
n = len(imseq);
# Imports to image with index = n
for x in range(0, n):
    #im_norm.append(imseq[x]+abs(imseq[x].min())+1)
    #Normalize image pixel values to between 1 and 65536 in a non retarded way
    im_norm.append(skie.rescale_intensity(imseq[x], in_range = (1, 65536)))
'''

In [7]:
imseq = seq_import('Melt_Kinetic_Run_18_127-', '.tif')
pimseq = [] # Empty list for padded image sequence
# Pad images with 50 pixel border of zeros
for i in range (0, len(imseq)):
    pimseq.append(np.pad(imseq[i], (50,50), 'constant', constant_values=(0,0)))

In [31]:
plt.imshow(pimseq[1])
plt.show()

In [12]:
# Check bulk growth 
## Radial average function, pretend center is at (1024,1024) for now
# Actual beam center is (1056, 1034) by manual calibration
def radial_profile(data, center):
    y, x = np.indices((data.shape))
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    r = r.astype(np.int)

    tbin = np.bincount(r.ravel(), data.ravel())
    nr = np.bincount(r.ravel())
    radialprofile = tbin / nr
    return radialprofile 

integrated_patterns = []
for i in range (0, len(imseq)):
    integrated_patterns.append(radial_profile(imseq[i], (1056, 1034)))

# SVD of radial averages 

from scipy import linalg
U, s, Vh = linalg.svd(integrated_patterns)

In [10]:
plt.plot(U[:,2])
plt.show()

In [14]:
# Difference of gaussian method
from matplotlib import pyplot as plt
from skimage import data
from skimage.feature import blob_dog#, blob_doh, blob_log
from skimage import io
from math import sqrt
from skimage.color import rgb2gray

image = him
image_gray = image/np.max(image)

# Too small of a threshold will make program fail to run
blobs_dog = blob_dog(image_gray, min_sigma = 0.25, max_sigma=10, sigma_ratio=4, threshold=0.03) 
# Compute radii in the 3rd column.
blobs_dog[:, 2] = blobs_dog[:, 2] * sqrt(2)

# Plot Blob DoG
null = [] #empty list
blobs_list = [blobs_dog, null]
colors = ['red', 'yellow']
titles = ['Peaks located by difference of gaussians method', 'Diffraction Pattern']
sequence = zip(blobs_list, colors, titles)

fig,axes = plt.subplots(1,2, sharex=True, sharey=True, subplot_kw={'adjustable':'box-forced'})
axes = axes.ravel()
for blobs, color, title in sequence:
    ax = axes[0]
    axes = axes[1:]
    ax.set_title(title)
    ax.imshow(image, interpolation='nearest', cmap = 'BuGn')
    for blob in blobs:
        y, x, r = blob
        c = plt.Circle((x, y), r, color=color, linewidth=2, fill=False)
        ax.add_patch(c)

plt.show()

In [15]:
len(blobs_dog)

1422

In [16]:
from skimage.draw import circle, circle_perimeter
# Needs skimage.draw
# Returns spot intensity
# Pass x, y position, radius (r), and some array (image)
def spot_intensity(x, y, r, image):
    rad = int(np.ceil(r)) # Returns nearest largest integer to provided radius
    x_pos = int(x) # Gets integer value for provided x
    y_pos = int(y) # Gets integer value for provided y
    out = 0 # Output variable
    
    rr, cc = circle(x_pos, y_pos, r-1) # Generates a list of points in circle, (r+1 because function used doesn't provide points on perimeter)
    
    for i in range (0, len(rr)):
        out += image[rr[i]][cc[i]]
    return out


In [17]:
plt.imshow(np.log(lf[0]))
plt.show()

C:\Users\User\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: RuntimeWarning: divide by zero encountered in log
  if __name__ == '__main__':
C:\Users\User\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: RuntimeWarning: invalid value encountered in log
  if __name__ == '__main__':


In [ ]:
blobs_dog[300]

In [ ]:
spot_intensity(1024, 1024, 1024, lf[0])

In [ ]:
out = []
for i in range (0, len(imseq)):
    out.append(spot_intensity(806, 1693, 5, imseq[i]))

In [ ]:
plt.plot(out, 'k.')
plt.plot(out, 'b')
plt.show()

In [18]:
# Shitfuckton of functions

from skimage.draw import circle, circle_perimeter
# Takes value (val) and checks against dataset (d_set) and checks if it's an outlier
# Returns either value or median of data set if outlier
# Outlier check based on whether or not value is within 3 sigma of mean
# Sigma is approximated by 1.4826*(Median absolute deviation)
# Needs numpy

def is_outlier(val, d_set):
    med = np.median(d_set) # Get median
    #mad = np.median(np.subtract(d_set, med)) # Get median absolute deviation
    mad = np.median(np.abs(d_set- med)) # Get median absolute deviation
    asigma = np.abs(mad*3*1.4826) # Absolute value of approximate sigma
    if (med-asigma) < val < (med+asigma):
        return val
    return med

# Pad dataset before filter based on window provided
# Give data set (d_set), and window width (x)
def winpad(d_set, x):
    if (x % 2 == 0): # Check if window is even
        raise Exception('Window width must be odd!')
    rad = np.int(np.floor(x/2)) # Calculate radius of window given
    pad = np.pad(d_set, rad, 'edge')
    return pad

# Hampel filter, input dataset (dset) and window width (x)
# Input an odd window or the window will be asymmetric and something fucks up
def hampel(d_set, x):
    if (x % 2 == 0): # Check if window is even
        raise Exception('Window width must be odd!')
    out = []
    temp = winpad(d_set, x)
    rad = np.int(np.floor(x/2))
    for i in range (0, len(d_set)):
        out.append(is_outlier(d_set[i], temp[i:i+x]))
    return out
# Return intensities all peaks provided in a given image
# Provide image (im), and list of peaks (plist) in form ((x1, y1, radius1), (x2, y2, radius2), ...)
# Peaks are assumed to be circular

def image_peak(im, plist):
    intensity = [] # Empty list for peak intensities
    for i in range (0, len(plist)):
        x = plist[i][0] # Get x position for first peak in list
        y = plist[i][1] # Get y position for first peak in list
        r = plist[i][2] # Get radius for first peak in list
        intensity.append(spot_intensity(x, y, r, im))
    return intensity

# Needs skimage.draw
# Returns spot intensity
# Pass x, y position, radius (r), and some array (image)
def spot_intensity(x, y, r, image):
    rad = int(np.ceil(r)) # Returns nearest largest integer to provided radius
    x_pos = int(x) # Gets integer value for provided x
    y_pos = int(y) # Gets integer value for provided y
    out = 0 # Output variable
    
    rr, cc = circle(x_pos, y_pos, r-1) # Generates a list of points in circle, (r+1 because function used doesn't provide points on perimeter)
    
    for i in range (0, len(rr)):
        out += image[rr[i]][cc[i]]
    return out

# Return intensities of all peaks provided over a list of images
def all_spot_intensities(im_list, plist):
    frame_intensities = [] # Empty list for peak intensities over a frame
    for i in range (0, len(im_list)):
        frame_intensities.append(image_peak(im_list[i], plist))
    return np.transpose(frame_intensities)

# Returns moving average of data set with window of n width
def moving_average(data, n):
    return np.convolve(data, np.ones((n,))/n)[(n-1):]

# Returns last n% of a given time series
# Returns x and y values
# X values are just based on position in series
def last_per(data, n):
    xval = [] # Empty list for x values 
    perc = 1-(n/100)
    maxval = len(data) # Max 
    minval = np.int(maxval*perc)
    
    yval = data[minval:maxval]
    xval = list(range(minval,maxval))
    
    return xval,yval

# Returns values between n% and m% of a given time series
# Returns x and y values
# X values are just based on position in series
def per_range(data, n, m):
    xval = [] # Empty list for x values 
    perc = (n/100)
    perc2 = (m/100)
    length = len(data) # length of data set
    maxval = np.int(length*perc2) # Index corresponding to m%
    minval = np.int(length*perc) # Index corresponding to n%
    
    yval = data[minval:maxval]
    xval = list(range(minval,maxval))
    
    return xval,yval

# Returns a set of points that correspond to some linear baseline 
# Linear baseline is determined by fitting the last n% of input time series
def linear_baseline(data, n):
    xval,yval = last_per(data, n) # Get last n percent of data set
    lm = np.polyfit(xval, yval, 1) # Fit to first degree polynomial
    lin = [] # Empty list for points generated from linear model
    for i in range (0, len(data)): # Generate list of points the length of data set with linear model
        lin.append(lm[0]*i) # Remove y-intercept
    return lin

# Returns a set of points that correspond to some linear baseline 
# Linear baseline is determined by fitting n-m% of input time series
def linear_baseline_range(data, n, m):
    xval,yval = per_range(data, n, m) # Get last n percent of data set
    lm = np.polyfit(xval, yval, 1) # Fit to first degree polynomial
    lin = [] # Empty list for points generated from linear model
    for i in range (0, len(data)): # Generate list of points the length of data set with linear model
        lin.append(lm[0]*i) # Remove y-intercept
    return lin

# Perform a baseline subtraction
# Subtracts a linear baseline that corresponds to the fit of the last n% of the input data set
def sub_linear_baseline(data, n):
    subdat = np.subtract(data, linear_baseline(data, n)) # Subtracts linear baseline 
    return subdat 

# Perform a baseline subtraction after applying a moving average of window width j to the input data
# Subtracts a linear baseline that corresponds to the fit of n-m% of the input data set
def sub_linear_baseline_av(data, n, m, j):
    av_dat = moving_average(data, j) # Moving average of data
    subdat = np.subtract(data, linear_baseline_range(av_dat, n, m)) # Subtracts linear baseline 
    return subdat 

# Perform a baseline subtraction
# Subtracts a linear baseline that corresponds to the fit of the first n% of the input data set
def sub_linear_baseline_first(data, n):
    subdat = np.subtract(data, linear_baseline_range(data, 0, n)) # Subtracts linear baseline 
    return subdat 

In [ ]:
# Normalizes to intensity seen at average of frames between 80% and 90% of the total run
# Requires input of total number of frames 
#def normalize(x, k):
#    return x/np.mean(x[int(np.ceil((k-1)*0.8)):int(np.ceil((k-1)*0.9))])

# Shifts baseline to average of first 30 points
#def bshift(x):
#    return x-np.mean(x[1:30])

# Avrami fit model
#def avrami(x, k, t):
#    return 1-np.exp(-np.power(k*(x-t), 3))

In [19]:
# Returns average of first n% of a given time series
def mean_first_per(data, n):
    perc = n/100 # Gets percentage corresponding to n%
    maxval = np.int(len(data)*perc) # Gets highest index value to stop at
    
    return np.mean(data[0:maxval])
# Returns average of last n% of a given time series
def mean_last_per(data, n):
    length = len(data)
    perc = 1-(n/100) # Gets percentage corresponding to n%
    minval = np.int(length*perc) # Gets highest index value to stop at
    
    return np.mean(data[minval:length])
# Shifts baseline to 0
# Input data list (data), percentage to use as baseline (n)
# Assumes average of first n% corresponds to 0%
def base_shift(data, n):
    datmean = mean_first_per(data, n)
    return np.subtract(data, datmean)

# Normalizes data set
# Input data list (data), and last percentage of data set to use as complete (n)
def normalize(data, n):
    datmean = mean_last_per(data, n)
    return np.divide(data, datmean)

# Normalizes data set to moving average of data set
# Input data list (data), percentage range of data set to use as complete (n-m), and moving average window (j)
def normalize_av(data, n, m, j):
    av_dat = moving_average(data, j) # Moving average of data set
    datmean = np.mean(per_range(data, n, m)[1]) # Average of range n-m% of moving average
    return np.divide(data, datmean)

# Corrects and normalizes an input growth curve by:
# Subtract linear baseline at end generated from moving average (assumes large irregular oscillations at end)
# Use range n - m % for linear baseline with window of j points for moving average
# Normalize to 1 using average of n - m% after baseline correction
# Undo linear baseline correction by subtracting baseline at beginning (assumes measurement of 0 is relatively stable)
# Use range 0 - i% for linear baseline at beginning
def curve_correction (data, n, m, j, i):
    p = base_shift(data, i) # Baseline shifts beginning 0 - i% to 0
    q = sub_linear_baseline_av(p, n, m, j) # Subtracts baseline generated from moving average in range n - m%, moving average window j points
    r = normalize_av(q, n, m, j) # Normalize by dividing by average of n - m% after baseline correction
    s = sub_linear_baseline_first(r, i) # Undo linear baseline correction by subtracting baseline from 0 - i%
    return s

In [20]:
from scipy.optimize import curve_fit
# Truncate values below some threshold value (n)
# Provide data list and threshold 
def truncate_n(dat, n):
    out = []
    for i in range (0, len(dat)):
        if dat[i] < n:
            out.append(dat[i])
        else:
            break
    return out
# Define avrami model as piecewise function 
# Function returns 0 for x < t (zero before t0)
# Function returns normal avrami model otherwise
def piecewise_avrami(x, k, t):
    n = 3 # Change if you want to change fit dimensionality
    if x < t: # Check if x < t
        return 0 # Return 0 if nucleation has not occured yet
    else:
        return 1-np.exp(-np.power(k*(x-t), n)) # Normal avrami model
# Vectorized version of avrami model
# Use this when fitting
def vect_p_av(x ,k):
    j = np.zeros(x.shape)
    for i in range(0, len(j)):
        j[i] = piecewise_avrami(x[i], k, 670)
    return j    
# Define some polynomial function
# Function returns 0 for x < t (zero before t0)
# Function returns polynomial otherwise
def polynomial(x, k, t):
    n = 3 # Change if you want to change dimensionality
    if x < t: # Check if x < t
        return 0 # Return 0 if nucleation has not occured yet
    else:
        return np.power((k*(x-t)), n)
# Vectorized polynomial for fitting 
def vect_polynomial(x, k, t):
    j = np.zeros(x.shape)
    for i in range (0, len(j)):
        j[i] = polynomial(x[i], k, t)
    return j

# Define some nth order polynomial function
def npolynomial(x, k):
    n = 3 # Change if you want to change dimensionality
    return np.power((k*(x)), n)
# Vectorized polynomial for fitting 
def vect_npolynomial(x, k):
    j = np.zeros(x.shape)
    for i in range (0, len(j)):
        j[i] = npolynomial(x[i], k)
    return j
# Fit input data set to polynomial model
# Biases fit so that it passes through last point and 0,0
def fit_to_polynomial(dat):
    x = list(range(0, len(dat))) # X values for fit
    y = list(dat) # Y values for fit (input data)
    for i in range (0, 1000): # Bias fit so that it always passes through the final point and 0,0
        x.append(x[-1])
        x.insert(0,0) 
        y.append(y[-1])
        y.insert(0,0)
    best_vals, covar = curve_fit(vect_polynomial, x, y)
    return best_vals

In [21]:
# Check these functions 

# Return intensities all peaks provided in a given image
# Provide image (im), and list of peaks (plist) in form ((x1, y1, radius1), (x2, y2, radius2), ...)
# Peaks are assumed to be circular
def image_peak(im, plist):
    intensity = [] # Empty list for peak intensities
    for i in range (0, len(plist)):
        x = plist[i][0] # Get x position for first peak in list
        y = plist[i][1] # Get y position for first peak in list
        r = plist[i][2] # Get radius for first peak in list
        intensity.append(spot_intensity(x, y, r, im))
    return intensity
# Needs skimage.draw
# Returns spot intensity
# Pass x, y position, radius (r), and some array (image)
def spot_intensity(x, y, r, image):
    rad = int(np.ceil(r)) # Returns nearest largest integer to provided radius
    x_pos = int(x) # Gets integer value for provided x
    y_pos = int(y) # Gets integer value for provided y
    out = 0 # Output variable
    
    rr, cc = circle(x_pos, y_pos, r-1) # Generates a list of points in circle, (r+1 because function used doesn't provide points on perimeter)
    
    for i in range (0, len(rr)):
        out += image[rr[i]][cc[i]]
    return out

# Return intensities of all peaks provided over a list of images
def all_spot_intensities(im_list, plist):
    frame_intensities = [] # Empty list for peak intensities over a frame
    for i in range (0, len(im_list)):
        frame_intensities.append(image_peak(im_list[i], plist))
    return np.transpose(frame_intensities)

In [22]:
# Pad x and y positions of blobs_dog list to account for border added to images
pblobs_dog = []
for i in range (0, len(blobs_dog)):
    temp = blobs_dog[i] # Temp variable to hold current blobs_dog 
    pblobs_dog.append([temp[0]+50, temp[1]+50, temp[2]]) # Append blobs_dog list with padded x,y values and peak radius

In [23]:
peaks = all_spot_intensities(pimseq, pblobs_dog)

In [ ]:
# Truncate peaks at 100 s
#trunc_peaks = []
#for i in range (0, len(peaks)):
#    trunc_peaks.append(peaks[i][1:100])

In [24]:

delay = 6 # Time furnace blocked beam for
tpeaks = [] # List for peaks with first n+1 seconds removed (where furnace blocked beam)
for i in range (0, len(peaks)):
    tpeaks.append(peaks[i][delay:-1]) 

In [ ]:
# Spikes not present so this is unnecessary 
#hpeaks = [] # List for peak growth curves with applied hampel filter
#for i in range (0, len(trunc_peaks)):
#    hpeaks.append(hampel(trunc_peaks[i], 3)) # Apply hampel filter with window width 3 to remove spikes

In [45]:
# Edge pad start of curves with 50 points 
# Only needed if crystallization starts very early
#ppeaks = [] # List for peak growth curves with edge padding
#for i in range(0, len(tpeaks)):
#    ppeaks.append(np.pad(tpeaks[i], (50, 0), 'edge'))


In [ ]:
# peak 300 for testing
#test = [365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 365.69999909, 349.69999778, 414.70000029, 337.70000052, 341.69999854, 334.69999933, 372.7000012 , 376.69999933, 393.70000035, 367.69999819, 500.69999631, 1092.70000303, 1656.70001239, 2485.69998517, 3349.69999528, 3707.69998538, 3996.69998002, 4074.6999824 , 4170.69998515, 4077.69998036, 4372.69997042, 4175.69998503, 4126.69997245, 4171.69997251, 4103.69997248, 4071.6999687 , 4157.69998628, 3925.69999045, 4016.69999659, 4182.69997115, 4205.6999861 , 4145.69998601, 4196.69999349, 4089.69999301, 4163.69996989, 4068.69998807, 3952.69999117, 4126.69998264, 4189.69996589, 4130.69996786, 4279.69996877, 4245.69996452, 4327.69996938, 4450.70000041, 4317.69996927, 4419.69996858, 4378.6999715 , 4283.69996872, 4079.69997287, 4217.69997025, 4229.69997305, 4279.69997287, 4375.69997227, 4289.69997698, 4159.69998521, 4355.70000529, 4341.70000279, 4538.70003414, 4552.7000351 , 4318.69998851, 4344.70000137, 4160.69996858, 4330.70000541, 4300.6999712 , 4337.70000577, 4436.70000339, 4330.69996977, 4222.69996536, 4488.70000023, 4457.69999588, 4289.69996834, 4441.69996929, 4463.69998145, 4385.69997197, 4456.69997323, 4503.69997287, 4441.69997311, 4452.69997394, 4409.69997182, 4485.70000362, 4638.70000426, 4591.70000541, 4551.70000607, 4530.70003337, 4430.69999909, 4692.70002985, 4467.69999599, 4675.69999814, 4569.69999814, 4317.69999588, 4678.69999111, 4490.6999948 , 4591.69999552, 4595.69999373, 4717.69999194, 4538.69999635, 4648.70000205, 4541.70000243, 4649.70000696, 4442.69997281]

In [28]:
plt.plot(peaks[170], 'r', tpeaks[300], 'k')
plt.show()

In [29]:
import  matplotlib.pyplot as plt
plt.plot(normalize_av(curve_correction(tpeaks[170], 70,75,100, 30),70,90,30))

plt.ylabel("Intensity (arbitrary units)")
plt.xlabel("Time (s)")
plt.show()

In [30]:
# Corrects and normalizes an input growth curve by:
# Subtract linear baseline at end generated from moving average (assumes large irregular oscillations at end)
# Use range n - m % for linear baseline with window of j points for moving average
# Normalize to 1 using average of n - m% after baseline correction
# Undo linear baseline correction by subtracting baseline at beginning (assumes measurement of 0 is relatively stable)
# Use range 0 - i% for linear baseline at beginning
# curve_correction(dat, n,m,j,i)
# Corrects growth curves with linear baseline subtraction at end between 70 and 90%
# Moving average window of 50 points
# Initial zero baseline from 0 to 1%

# Causes a crash if the moving average window is too large relative to the data set
cpeaks = []
for i in range (0, len(tpeaks)):
    cpeaks.append(normalize_av(curve_correction(tpeaks[i], 70, 75, 100, 30), 70,90,30))

C:\Users\User\Anaconda3\lib\site-packages\ipykernel\__main__.py:32: RuntimeWarning: invalid value encountered in true_divide


In [31]:
plt.plot(cpeaks[190])
plt.show()

In [32]:
# Return index for when value exceeds some threshold value (n)
# Moving average with window of j points is applied
def get_n(dat, n, j):
    av_dat = moving_average(dat, j)
    out = []
    for i in range (0, len(dat)):
        if dat[i] < n:
            out.append(i)
        else:
            break
    return out[-1]

In [33]:
import scipy.signal as signal
import matplotlib.pyplot as plt
 
# fdat = [0 for x in range (0, len(blobs_dog))] # Creates empty list for filtered data
# Design of filter
N  = 4    # Filter order
Wn = 0.5  # Cutoff frequency as a fraction of sampling rate
# Butterworth filter
B, A = signal.butter(N, Wn, output='ba')

In [34]:
fdat = [] # Empty list for filtered data
for i in range (0, len(cpeaks)):
    fdat.append(signal.filtfilt(B, A, cpeaks[i]))

In [35]:
plt.plot(fdat[190])
plt.show()

In [36]:
tdat = []
for i in range (0, len(fdat)):
    tdat.append(truncate_n(fdat[i], 0.2))

In [37]:
f_params = [] # Empty list for fit parameters

for i in range (0, len(tdat)):
    if len(tdat[i])<3:
        f_params.append([0,0])
    else:
        try:
            best_vals = fit_to_polynomial(tdat[i])
        except RuntimeError:
            best_vals = [0,0]
        f_params.append(best_vals)
    

C:\Users\User\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:690: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


In [38]:
plt.plot(np.transpose(f_params)[1], np.transpose(f_params)[0], 'r.')
plt.show()

In [41]:
rparams = [] # Parameters with [0,0] removed
for i in range (0, len(f_params)):
    if f_params[i][1] > 0: # Only take fit parameters if t0 is > 0
        if f_params[i][0] > 0: # Only take fit parameters if k is > 0
            rparams.append([f_params[i][0], f_params[i][1], i])

In [42]:
plt.plot(np.transpose(rparams)[1], np.transpose(rparams)[0], 'k.')
plt.show()

In [43]:
plt.plot(cpeaks[33])
plt.show()

In [44]:
check = [] # Empty list to check for points interest 
for i in range (0, len(rparams)):
    #if rparams[i][0] > 1: # check for k values 
    if 30 < rparams[i][1] < 40 : # check for t0 values 
        check.append(rparams[i])

In [45]:
plt.plot(cpeaks[5],marker = '.')
plt.show()

In [46]:
#np.sort(check,-2)
check

[[0.47647188282352027, 37.998294413645965, 19],
 [0.46584351487277159, 37.997630642611561, 25],
 [0.44915644961701495, 37.998306454605498, 31],
 [0.10901598260265172, 37.744903220379626, 41],
 [0.066746594124261135, 35.083602226988127, 57],
 [0.034103632936270861, 34.628602450158155, 60],
 [0.028300426229470963, 35.145781330283029, 63],
 [0.11626508355256997, 39.021036843263182, 70],
 [0.074638200871600321, 37.025053976968316, 82],
 [0.29772665518264946, 38.184504856191815, 86],
 [0.044293866106966576, 35.144163164103546, 91],
 [0.084024489136313177, 37.213837742626616, 96],
 [0.13195728630496228, 39.919921623302599, 100],
 [0.03730767732846204, 33.795222110384124, 102],
 [0.060512942984424931, 35.391189385763475, 108],
 [0.35633531338680019, 37.000125589405648, 111],
 [0.12700032014815429, 31.798054468527194, 113],
 [0.18728759120120098, 36.037879031665504, 115],
 [0.058698982658799123, 38.204745037658455, 117],
 [0.22687509718447918, 39.706901474077128, 120],
 [0.43476634001454406, 3

In [47]:
rparams2 = [] # Parameters with [0,0] removed
for i in range (0, len(rparams)):
    if rparams[i][1] > 30: # Only take fit parameters if t0 is > 30 (manually found value where curves are nonsense)
        rparams2.append(rparams[i])

In [51]:
plt.plot(np.transpose(rparams2)[1], np.transpose(rparams2)[0], 'k.')
plt.xlabel("t0 value")
plt.ylabel("k value")
plt.title("Melt kinetic run 18, T = 127 K")
plt.show()

In [54]:
plt.plot(np.transpose(rparams2)[0], np.transpose(rparams2)[1], 'k.')
plt.ylabel("t0 value")
plt.xlabel("k value")
plt.title("Melt kinetic run 18, T = 127 K")
plt.show()

In [55]:
np.savetxt("k, t0, peak index.csv", rparams2)

In [217]:
plt.plot(cpeaks[753])
plt.show()

In [67]:
np.mean(np.transpose(rparams2)[0])

0.60684262588491078

In [72]:
np.min(np.transpose(rparams2)[1])-1

1.5036262020725681

In [227]:
plt.hist(np.transpose(rparams)[1], bins='auto')  # arguments are passed to np.histogram
plt.title("Histogram with 'auto' bins")
plt.show()

In [191]:
a = []
for i in range (0, len(rparams2)):
    if rparams2[i][0] < 0.7:
        a.append(rparams2[i][0])
plt.hist(a, bins = 'auto')
plt.title('histogram of k values')
plt.ylabel('counts')
plt.xlabel('k value bins')
plt.show()

In [198]:
a = []
for i in range (0, len(rparams2)):
    if rparams2[i][1] > 10:
        a.append(rparams2[i][1])
plt.hist(a, bins = 'auto')
plt.title('histogram of t0 values')
plt.ylabel('counts')
plt.xlabel('t0 value bins')
plt.show()

[]

In [77]:
# Plot Blob DoG
null = [] #empty list
blobs_list = [blobs_dog, null]
colors = ['red', 'yellow']
titles = ['Peaks located by difference of gaussians method', 'Diffraction Pattern']
sequence = zip(blobs_list, colors, titles)

fig,axes = plt.subplots(1,2, sharex=True, sharey=True, subplot_kw={'adjustable':'box-forced'})
axes = axes.ravel()
for blobs, color, title in sequence:
    ax = axes[0]
    axes = axes[1:]
    ax.set_title(title)
    ax.imshow(image, interpolation='nearest', cmap = 'BuGn')
    for blob in blobs:
        y, x, r = blob
        c = plt.Circle((x, y), r, color=color, linewidth=2, fill=False)
        ax.add_patch(c)

plt.show()

In [237]:
# Check bulk growth 
## Radial average function, pretend center is at (1024,1024) for now
def radial_profile(data, center):
    y, x = np.indices((data.shape))
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    r = r.astype(np.int)

    tbin = np.bincount(r.ravel(), data.ravel())
    nr = np.bincount(r.ravel())
    radialprofile = tbin / nr
    return radialprofile 

integrated_patterns = []
for i in range (0, len(imseq)):
    integrated_patterns.append(radial_profile(imseq[i], (1024,1024)))

# SVD of radial averages 

from scipy import linalg
U, s, Vh = linalg.svd(integrated_patterns)

plt.plot(U[:,1], marker = '.')
plt.show()

In [201]:
bulk = [-0.010618033, -0.010702187, -0.010045366, -0.010469013, -0.010005388, -0.009453031, -0.009602459, -0.009374603, -0.009509486, -0.009262173, -0.00868389, -0.009232696, -0.008968024, -0.009089219, -0.008391475, -0.008583572, -0.008367971, -0.008397161, -0.008458041, -0.007760815, -0.00810188, -0.008243236, -0.007451357, -0.008209506, -0.008234447, -0.008407837, -0.007588457, -0.007617142, -0.007765034, -0.008227437, -0.008137681, -0.007112073, -0.007771342, -0.007532717, -0.007603996, -0.007950837, -0.007600917, -0.007796662, -0.007382528, -0.007502155, -0.00697404, -0.007331408, -0.006385506, -0.005832081, -0.005932957, -0.004864814, -0.003898141, -0.002836274, -0.001970251, -0.001133124, -0.000442634, 2.31969E-05, 0.000176559, 0.000881155, 0.001174823, 0.001820202, 0.003057335, 0.003921401, 0.004231023, 0.004398804, 0.004665224, 0.004748712, 0.004847198, 0.00450095, 0.004479236, 0.00453717, 0.003975428, 0.004066963, 0.004382726, 0.0040672, 0.004277923, 0.00437703, 0.004062581, 0.004041599, 0.003800643, 0.004014245, 0.00411727, 0.003936493, 0.00380549, 0.004117341, 0.004191951, 0.004096645, 0.003853682, 0.004124873, 0.004128822, 0.004582453, 0.004522232, 0.003984165, 0.004184808, 0.00441592, 0.004385197, 0.004541383, 0.004305236, 0.004401508, 0.004217874, 0.003920547, 0.004128345, 0.003954468, 0.0038997, 0.004053762, 0.003932482, 0.004031586, 0.004002506, 0.003921699, 0.003659427, 0.004124206, 0.004409776, 0.004200089, 0.004216551, 0.004320405, 0.004696201, 0.004380384, 0.004512264, 0.004606207, 0.004350901, 0.004585776, 0.004703557, 0.004467891, 0.00439866, 0.004135496, 0.003984516, 0.003767038, 0.003956429, 0.003948045, 0.003919512, 0.004096511, 0.003658474, 0.003891851, 0.003959506, 0.003837453, 0.004078071, 0.004280875, 0.004343448, 0.004596779, 0.00420706, 0.004249367, 0.004529423, 0.004579673, 0.004568296, 0.004606573, 0.004507968, 0.004470949, 0.004140858, 0.003949683, 0.004179621, 0.004176694, 0.004184638, 0.003973141, 0.003940603, 0.003729126, 0.003927252, 0.003955651, 0.004020556, 0.004143972, 0.004216623, 0.004217783, 0.004348495, 0.004387274, 0.004443163, 0.004750391, 0.004898346, 0.004976769, 0.004672763, 0.004389419, 0.004333485, 0.004210702, 0.004381004, 0.004234156, 0.004545696, 0.004238226, 0.004611918, 0.004124514, 0.004037488, 0.003847894, 0.003893974, 0.004076606, 0.00439873, 0.004108438, 0.004446471, 0.004377558, 0.004375326, 0.004675803, 0.004601692, 0.005047089, 0.004765059, 0.004846701, 0.004916091, 0.004562777, 0.004623249, 0.004470267, 0.004353666, 0.004488304, 0.004277792, 0.004075936, 0.004243331, 0.004128465, 0.004342105, 0.004199832, 0.003896919, 0.003999837, 0.003905799, 0.00438519, 0.004103416, 0.004375078, 0.004225843, 0.004326288, 0.004607351, 0.004434692, 0.004576324, 0.004626515, 0.004763267, 0.004815867, 0.004714557, 0.004368797, 0.00413819, 0.004109015, 0.004053669, 0.004179573, 0.00415296, 0.004168377, 0.004229939, 0.004029786, 0.004104248, 0.003834388, 0.004257383, 0.00431446, 0.004055339, 0.004591951, 0.004270021, 0.004656744, 0.004688882, 0.004759256, 0.00437015, 0.004922858, 0.004571353, 0.004593981, 0.004268442, 0.004087842, 0.004041373, 0.004316509, 0.004152845, 0.003814349, 0.004024006, 0.004274772, 0.003978082, 0.00384612, 0.003937838, 0.00413322, 0.004162548, 0.004146885, 0.004035212, 0.004210077, 0.004313622, 0.004424646, 0.004569422, 0.004307931, 0.004393993, 0.004724371, 0.004534267, 0.004672972, 0.004045118, 0.004119859, 0.004081934, 0.003929577, 0.003837084, 0.003646804, 0.004010142, 0.003892546, 0.003823974, 0.003898795, 0.004062747, 0.004301551, 0.004078911, 0.004013285, 0.004328478, 0.004402881, 0.004531393, 0.004732512, 0.004542397, 0.00483768, 0.004369761, 0.00446889, 0.004588537, 0.004554786, 0.004730079, 0.004673937, 0.004301034, 0.003954225, 0.004028085, 0.003883434, 0.003792393, 0.004135406, 0.004028004, 0.003838379, 0.003689642, 0.004040606, 0.003974352, 0.004142337, 0.004118002, 0.003944275, 0.004501349, 0.004730117, 0.004979836, 0.00459259, 0.004666791, 0.004829314, 0.004609126, 0.004511763, 0.00430575, 0.004105118, 0.00420981, 0.004187259, 0.004184995, 0.003948721, 0.003979322, 0.004015653, 0.004055797, 0.003911243, 0.00376127, 0.00381297, 0.003861015, 0.00409009, 0.004171029, 0.004457091, 0.004380217, 0.004387912, 0.004514433, 0.004380495, 0.004619304, 0.004719461, 0.004566491, 0.005014346, 0.004530063, 0.00447204, 0.004604741, 0.003973992, 0.004173434, 0.003968399, 0.003901682, 0.003958696, 0.003587704, 0.003830964, 0.004160648, 0.003712335, 0.003876017, 0.004087356, 0.004212598, 0.003957131, 0.004016799, 0.004202452, 0.004569631, 0.004208436, 0.004540047, 0.004651086, 0.004683073, 0.004578707, 0.004241522, 0.004386056, 0.004047575, 0.004405374, 0.004168586, 0.003887555, 0.003769091, 0.003673857, 0.003843115, 0.003807227, 0.003648299, 0.003648615, 0.003964161, 0.003931397, 0.003884629, 0.004069158, 0.003901503, 0.003956251, 0.004162931, 0.004513348, 0.004475948, 0.004439141, 0.004362496, 0.004256619, 0.004552007, 0.004096439, 0.004201565, 0.004422109, 0.003909845, 0.003686485, 0.00352134, 0.003730876, 0.003556932, 0.003701936, 0.003575524, 0.003772897, 0.003778122, 0.003810325, 0.00417281, 0.004263909, 0.00436328, 0.004036994, 0.004640232, 0.004452742, 0.004474475, 0.004498833, 0.004600899, 0.004757743, 0.004649487, 0.004502478, 0.004618398, 0.003936839, 0.004308296, 0.004222947, 0.003693016, 0.003728972, 0.003746406, 0.003562752, 0.003594244, 0.003479867, 0.00359404, 0.003589836, 0.003565327, 0.003825194, 0.003805904, 0.003891383, 0.003784426, 0.004138525, 0.004202178, 0.004518747, 0.004314405, 0.004405213, 0.0042944, 0.004288121, 0.004480025, 0.004524121, 0.004379617, 0.00394369, 0.003802573, 0.003717103, 0.0040918, 0.003381368, 0.003470572, 0.003595196, 0.003840127, 0.00381128, 0.003665964, 0.003748317, 0.003827301, 0.003791421, 0.003659749, 0.004031797, 0.003966933, 0.004021431, 0.004227321, 0.004259957, 0.004484037, 0.004627314, 0.004567631, 0.004563959, 0.004440716, 0.003913222, 0.003799398, 0.004192732, 0.003583449, 0.003575966, 0.004006004, 0.003780666, 0.003543433, 0.003633395, 0.00405607, 0.003896938, 0.003753753, 0.003612462, 0.003962041, 0.003884229, 0.003974907, 0.00446137, 0.004496, 0.004687636, 0.004392955, 0.004617567, 0.004247412, 0.004442427, 0.004298673, 0.004032352, 0.003922597, 0.004281308, 0.00397154, 0.003730452, 0.003834379, 0.003593594, 0.003965488, 0.003576552, 0.004011622, 0.003995712, 0.004086291, 0.003947231, 0.003924868, 0.004318016, 0.004143846, 0.004335325, 0.004450509, 0.004648337, 0.005002843, 0.004855688, 0.004656919, 0.004815412, 0.004597241, 0.004270394, 0.004217779, 0.004100314, 0.004227442, 0.003835257, 0.003691444, 0.003727504, 0.003850607, 0.003263669, 0.003671487, 0.003697896, 0.003703743, 0.003982738, 0.004076839, 0.004246738, 0.004021167, 0.004001203, 0.00447109, 0.004543777, 0.004647185, 0.004634709, 0.004528026, 0.004690102, 0.004672743, 0.0045813, 0.004511352, 0.004307266, 0.004202064, 0.003933552, 0.003864666, 0.00379939, 0.003713377, 0.003965528, 0.003848198, 0.0040977, 0.003939316, 0.004072458, 0.003949772, 0.004242749, 0.004250676, 0.004294053, 0.004112664, 0.004531625, 0.004765821, 0.00490463, 0.00438927, 0.00460071, 0.004023953, 0.004167773, 0.004121317, 0.004057887, 0.004090334, 0.003735944, 0.003720523, 0.003632085, 0.003564655, 0.003244951, 0.00358253, 0.003305773, 0.003977206, 0.003918137, 0.003810902, 0.004103809, 0.004128361, 0.004158377, 0.004451082, 0.004313097, 0.004700793, 0.004414295, 0.004575677, 0.004631461, 0.004279425, 0.004441125, 0.004555665, 0.004343322, 0.004133931, 0.004334398, 0.003973861, 0.003828066, 0.003962046, 0.00359138, 0.003640021, 0.003515898, 0.003703549, 0.004060442, 0.003563495, 0.003820488, 0.003956709, 0.00376352, 0.004298582, 0.004284639, 0.004413835, 0.004690782, 0.004677184, 0.004446543, 0.004730424, 0.004579122, 0.004341587, 0.004382199, 0.004303129, 0.004032525, 0.003722854, 0.003848126, 0.003836233, 0.003499728, 0.003578373, 0.00358823, 0.003911789, 0.003785553, 0.003787052, 0.003893555, 0.003989708, 0.004217831, 0.004157708, 0.004638085, 0.004515029, 0.004840386, 0.004398912, 0.004531459, 0.004585435, 0.004992782, 0.004532883, 0.004536001, 0.004546942, 0.004639135, 0.00442139, 0.004186772, 0.004064014, 0.003903525, 0.003687659, 0.004058827, 0.003816962, 0.003907063, 0.003549077, 0.003952845, 0.004257115, 0.003934559, 0.004136627, 0.00389018, 0.004257592, 0.004386697, 0.004804508, 0.00439084, 0.004506863, 0.004375148, 0.004470276, 0.00461046, 0.004615741, 0.004029613, 0.003929904, 0.004184902, 0.003721142, 0.003943833, 0.00348434, 0.003819225, 0.003762883, 0.003728835, 0.003847673, 0.003757864, 0.003800926, 0.004283427, 0.00390682, 0.00404696, 0.004247219, 0.004492434, 0.004060132, 0.004567408, 0.004952655, 0.004789006, 0.004475326, 0.004763512, 0.004495917, 0.004616881, 0.004357374, 0.004323158, 0.003740058, 0.003943322, 0.003746941, 0.003782161, 0.003822159, 0.003774712, 0.003716322, 0.003882915, 0.00398861, 0.004021338, 0.004295842, 0.004038492, 0.004330217, 0.004749945, 0.004491599, 0.004763651, 0.004901127, 0.005041231, 0.004865391, 0.004862332, 0.004576551, 0.00478791, 0.004263369, 0.004458727, 0.004217291, 0.004011645, 0.00396956, 0.003924959, 0.003830744, 0.003856054, 0.003945242, 0.003695392, 0.003834791, 0.004057464, 0.00421692, 0.004086975, 0.004372145, 0.004573005, 0.004371987, 0.004372661, 0.004798246, 0.004653228, 0.00492831, 0.004490008, 0.004823206, 0.004547189, 0.004364119, 0.004424783, 0.004260228, 0.004173407, 0.004235745, 0.003753934, 0.003821093, 0.003873507, 0.003746436, 0.003585646, 0.003875481, 0.004068486, 0.004011013, 0.003777348, 0.004165403, 0.004367212, 0.004207363, 0.004143284, 0.00467436, 0.004794439, 0.004458508, 0.004952124, 0.004827911, 0.00481228, 0.004912745, 0.004726374, 0.004665608, 0.004029481, 0.00425902, 0.00396478, 0.004057586, 0.00399052, 0.003765279, 0.003534241, 0.004049961, 0.004069148, 0.00408266, 0.004078803, 0.003969246, 0.004203986, 0.004578312, 0.004649226, 0.004377515, 0.004839572, 0.00463655, 0.004846813, 0.00513346, 0.004732423, 0.004735058, 0.00466067, 0.004383504, 0.004110823, 0.004202648, 0.003927612, 0.003657352, 0.004142071, 0.003945275, 0.003945598, 0.004084624, 0.004219726, 0.003957996, 0.004030442, 0.004260595, 0.004253624, 0.004648218, 0.004318726, 0.004715241, 4.68351428e-03]

In [211]:
bulk = bulk[11:100]
plt.plot(bulk)
plt.show()

In [228]:
np.min(np.transpose(rparams2)[1])

30.095225983852984

In [206]:
plt.plot(bulk)
plt.show()

In [214]:
nbulk = normalize_av(np.subtract(bulk, -0.008), 60, 70, 10)
plt.plot(nbulk, 'k.')
plt.show()

In [229]:
plt.plot(nbulk[0:34])
plt.show()

In [275]:
q = fit_to_polynomial(np.divide(np.add(curve_correction(bulk, 70, 75, 100, 30), 0.015),0.55)[0:34])

In [276]:
aoeu = np.divide(np.add(curve_correction(bulk, 70, 75, 100, 30), 0.015),0.55)
plt.plot(aoeu)
plt.show()

In [277]:
a = []
for i in range (0, 100):
    a.append(piecewise_avrami(i, q[0], q[1]))

In [278]:
plt.plot(aoeu, 'k.', a, 'r')
plt.show()